In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from constants import *

In [4]:
NODE_1 = 237
NODE_2 = 229
NODE_3 = 212
NODE_4 = 144

ME = NODE_1
OTHERS = [NODE_2, NODE_3, NODE_4]

In [5]:
PREFIX = "../../data/10"
NODE = "4"
PACKETS = f"{PREFIX}/node-{NODE}/packets"

In [6]:
netwrok_trace_type_mapping = {
    'timestamp': (1, 'Int64'),
    'size': (2, 'Int32'),
    'src_ip': (3, 'Int16'),
    'dest_ip': (4, 'Int16'),
    'src_port': (5, 'Int32'),
    'dest_port': (6, 'Int32'),
}

In [7]:
def reload_df(f):
    flows_df = pd.read_csv(f, index_col=False)
    cols = flows_df.columns
    flows_df[cols] = flows_df[cols].apply(pd.to_numeric, errors='coerce')
    flows_df = flows_df.dropna()
    return flows_df

In [8]:
packets_df = pd.read_csv(f"{PREFIX}/node-{NODE}/packets", header=None, index_col=False, names=netwrok_trace_type_mapping.keys(), sep=',', on_bad_lines='skip')

In [9]:
packets_df = packets_df[(packets_df['src_ip'] == NODE_4) & (packets_df['dest_ip'].isin([NODE_1]))].sort_values(by='timestamp')
packets_df['timestamp'] = packets_df['timestamp'] - packets_df['timestamp'].min()
packets = packets_df[['timestamp', 'size']].values
start, end = packets_df['timestamp'].min(), packets_df['timestamp'].max()
step = 500 * NANO_TO_MICRO
steps_begin = np.arange(start, end, step)
steps_end = steps_begin + step
s = np.clip(np.searchsorted(steps_begin, packets[:, 0]), 0, len(steps_begin) - 1)

In [10]:
s1 = np.clip(np.searchsorted(packets[:, 0], steps_begin), 0, len(packets) - 1)
s2 = np.clip(np.searchsorted(packets[:, 0], steps_end), 0, len(packets) - 1)
sizecsum = np.insert(np.cumsum(packets[:, 1]), 0, 0)
vals = sizecsum[s2] - sizecsum[s1]
#vals[0] = sizecsum[(s2 - 1)[0]] - 0

In [11]:
last_step = steps_end[-1]

In [25]:
packets[:, 1][:5]

array([ 87, 239,  66,  87, 239], dtype=int64)

In [24]:
packets[:, 0][:5]

array([       0,    36210,  3650174, 11936047, 11996534], dtype=int64)

In [26]:
steps_begin[7]

3500000

In [27]:
steps_end[7]

4000000

In [16]:
s2

array([      2,       2,       2, ..., 7918846, 7918846, 7918847])

In [17]:
s1

array([      0,       2,       2, ..., 7918846, 7918846, 7918846])

In [20]:
np.argwhere(vals > 0)

array([[       0],
       [       7],
       [      23],
       ...,
       [27815396],
       [27815397],
       [27815401]])

In [21]:
s2[7], s1[7]

(3, 2)

In [28]:
vals[7]

66